# DTI Probabilistic Tractography

In the last tutorial, we show how one can perform deterministic tractography from DTI model. Deterministic tractography algorithms are, however, limited for highly depend on the errors of discrete local voxel direction estimates. To overcome this issue, probabilistic tracking generates streamlines by following directions sampled from probabilistic distribution functions.
In this tutorial, we show how one can perform probabilistic tractography from the orientation distribution estimated from DTI.

In this tutorial, we show how one can perform probabilistic tractography from the orientation distribution estimated from DTI.
In general, to perform probabilistic fibre tracking, we are needed:

1) A method for generating the probability distribution function of directions

2) As for deterministic tractography, a set of seeds from which to begin propagating streamlines

3) As for deterministic tractography, a set of stopping criteria for streamline propagation

4) As for deterministic tractography, a method for propagating streamlines

To begin, let's load the Stanford's dMRI dataset and its labels:

In [1]:
from dipy.core.gradients import gradient_table
from dipy.data import get_fnames
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti, load_nifti_data

hardi_fname, hardi_bval_fname, hardi_bvec_fname = get_fnames('stanford_hardi')

data, affine, hardi_img = load_nifti(hardi_fname, return_img=True)
bvals, bvecs = read_bvals_bvecs(hardi_bval_fname, hardi_bvec_fname)
gtab = gradient_table(bvals, bvecs)

label_fname = get_fnames('stanford_labels')
labels = load_nifti_data(label_fname)
white_matter = (labels == 1) | (labels == 2)

As for the previous examples, the loaded data is masked:

In [2]:
from dipy.segment.mask import median_otsu

maskdata, mask = median_otsu(data, vol_idx=range(0, 9),
                             numpass=1, dilate=5)

### 1) Getting the directions from DTI's orientation distribution function

The first thing we need to do is to generate the orientation distribution function from DTI. For this we need to first fit the DTI model to our data:

In [3]:
from dipy.reconst.dti import TensorModel

dti_model = TensorModel(gtab)
dti_fit = dti_model.fit(maskdata, mask=white_matter)

The orientation distribution function from dti can be calculated from the "dti_fit" atribute "odf". For this we also need to input directions evenly sampling the 3D cartesian spherical space. These directions are loaded from "dipy.data".

In [4]:
from dipy.data import get_sphere
sphere = get_sphere('repulsion724')

tensor_odfs = dti_fit.odf(sphere)

For quality assurance let's visualize the probability distribution function for each voxel:

In [5]:
from dipy.viz import window, actor

scene = window.Scene()

odf_actor = actor.odf_slicer(tensor_odfs, sphere=sphere, scale=0.5,
                             colormap='plasma')
scene.add(odf_actor)
print('Saving illustration as tensor_odfs.png')
window.record(scene, n_frames=1, out_path='tensor_odfs.png', size=(600, 600))
window.show(scene)

C:\Users\rafae\anaconda3\envs\dipy_release\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
C:\Users\rafae\anaconda3\envs\dipy_release\lib\site-packages\fury\actor.py:956: RuntimeWarning: invalid value encountered in true_divide
  m /= np.abs(m).max()


Saving illustration as tensor_odfs.png


To use these DTI's orientation distribution functions for probabilistic tracking we need to instantiate a "ProbabiticDirectionGetter" class object which can be imported from "dipy.direction".

Note: Input variable "max_angle" gives the maximum allowed angle between incoming direction and new direction in tractography propagation

In [6]:
from dipy.direction import ProbabilisticDirectionGetter

pmf = tensor_odfs.clip(min=0)
prob_dg = ProbabilisticDirectionGetter.from_pmf(pmf, max_angle=30.,
                                                sphere=sphere)

### 2 & 3) Defining the seeds and stopping criterion

As for the deterministic tracking example, we seed the voxels in corpus callosum sagittal slice with a grid density of 2 × 2 × 2. Streamlines will stop propagating when reaching FA values lower than 0.2.

In [7]:
from dipy.tracking import utils

seed_mask = (labels == 2)
seeds = utils.seeds_from_mask(seed_mask, affine, density=[2, 2, 2])

In [8]:
from dipy.tracking.stopping_criterion import ThresholdStoppingCriterion

fa = dti_fit.fa
stopping_criterion = ThresholdStoppingCriterion(fa, .2)

### 4) Propagating the streamlines

As for the previous tutorial, we performed tracking using the Dipy's "LocalTracking" procedure: 

In [9]:
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.streamline import Streamlines

streamline_generator = LocalTracking(prob_dg, stopping_criterion, seeds,
                                     affine, step_size=.5)
streamlines = Streamlines(streamline_generator)

Below we display the resulting streamlines using the fury python package.

In [10]:
from dipy.viz import window, actor, has_fury
from dipy.viz import colormap

if has_fury:
    # Prepare the display objects.
    color = colormap.line_colors(streamlines)

    streamlines_actor = actor.line(streamlines,
                                   colormap.line_colors(streamlines))

    # Create the 3D display.
    scene = window.Scene()
    scene.add(streamlines_actor)

    # Save still images for this static example. Or for interactivity use
    window.record(scene, out_path='dti_tractogram.png', size=(800, 800))
    window.show(scene)

We also save the streamlines as a Trackvis file.

In [11]:
from dipy.io.stateful_tractogram import Space, StatefulTractogram
from dipy.io.streamline import save_trk

sft = StatefulTractogram(streamlines, hardi_img, Space.RASMM)
save_trk(sft, "dti_CC_tractography_propabilistic.trk", streamlines)